In [2]:
import math
import numpy as np
import pandas as pd

import scikitplot
import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils

In [3]:
# Adicionando a path das pastas

train_dir = "./images/train" #passing the path with training images
test_dir = "./images/validation"   #passing the path with testing images

In [4]:
#original size of the image
img_size = 48 

In [5]:
# processando o pre training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [6]:
# processando o pre testing set
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                         validation_split = 0.2)

In [7]:
# depois tem que alterar pra binary pra ver qual performs better
train_set = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (img_size,img_size),
                                                    batch_size = 32,
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical",
                                                    subset = "training"
                                                   )


Found 28821 images belonging to 7 classes.


In [8]:
validation_set = validation_datagen.flow_from_directory( directory = test_dir,
                                                              target_size = (img_size,img_size),
                                                              batch_size = 32,
                                                              color_mode = "grayscale",
                                                              class_mode = "categorical",
                                                              subset = "validation"
                                                             )

Found 1411 images belonging to 7 classes.


In [9]:
# num_classes usado na hora do dense
num_classes = 7

In [10]:
# Inicializing the cnn
cnn = Sequential()

# First Layer 
cnn.add(
    Conv2D(
        filters = 64,
        kernel_size=(5,5),
        input_shape=(img_size, img_size, 1),
        activation = 'elu',
        name = 'Conv2D_1'
    )
)

cnn.add(BatchNormalization(name='batchnorm_1'))
    
# Second Layer
cnn.add(
        Conv2D(
            filters=64,
            kernel_size=(5,5),
            activation='elu',
            name='conv2d_2'
        )
    )

cnn.add(BatchNormalization(name='batchnorm_2'))

cnn.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_1'))
cnn.add(Dropout(0.4, name='dropout_1'))

# Third Layer
cnn.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            name='conv2d_3'
        )
    )

cnn.add(BatchNormalization(name='batchnorm_3'))
    
# Fouth Layer
cnn.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            name='conv2d_4'
        )
    )
cnn.add(BatchNormalization(name='batchnorm_4'))

cnn.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_2'))
cnn.add(Dropout(0.4, name='dropout_2'))


# flattening
cnn.add(Flatten(name='flatten'))
    
# densing
cnn.add(
    Dense(
        128,
        activation='elu',
        kernel_initializer='he_normal',
        name='dense_1'
    )
)
cnn.add(BatchNormalization(name='batchnorm_7'))

cnn.add(Dropout(0.6, name='dropout_4'))

cnn.add(
    Dense(
        num_classes,
        activation='softmax',
        name='out_layer'
    )
)

cnn.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

In [11]:
# usando GPU pode aumentar a qntd de epochs
epochs = 25
# dizem que batch size = 32 funciona melhor, mas vi mtts com 64
batch_size = 32

In [12]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_1 (Conv2D)            (None, 44, 44, 64)        1664      
_________________________________________________________________
batchnorm_1 (BatchNormalizat (None, 44, 44, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 40, 40, 64)        102464    
_________________________________________________________________
batchnorm_2 (BatchNormalizat (None, 40, 40, 64)        256       
_________________________________________________________________
maxpool2d_1 (MaxPooling2D)   (None, 20, 20, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 128)       7

In [13]:
# training the cnn

history = cnn.fit(x = train_set,epochs = epochs,validation_data = validation_set)

Epoch 1/25
901/901 [==============================] - 469s 518ms/step - loss: 2.0158 - accuracy: 0.2938 - val_loss: 1.5289 - val_accuracy: 0.4096
Epoch 2/25
901/901 [==============================] - 465s 516ms/step - loss: 1.5219 - accuracy: 0.4158 - val_loss: 1.4922 - val_accuracy: 0.4203
Epoch 3/25
901/901 [==============================] - 471s 522ms/step - loss: 1.4344 - accuracy: 0.4495 - val_loss: 1.4644 - val_accuracy: 0.4444
Epoch 4/25
901/901 [==============================] - 469s 521ms/step - loss: 1.3825 - accuracy: 0.4709 - val_loss: 1.2730 - val_accuracy: 0.5138
Epoch 5/25
901/901 [==============================] - 463s 514ms/step - loss: 1.3354 - accuracy: 0.4889 - val_loss: 1.2706 - val_accuracy: 0.5237
Epoch 6/25
901/901 [==============================] - 484s 537ms/step - loss: 1.3039 - accuracy: 0.5071 - val_loss: 1.3282 - val_accuracy: 0.5273
Epoch 7/25
901/901 [==============================] - 522s 579ms/step - loss: 1.2709 - accuracy: 0.5194 - val_loss: 1.2256 -

In [14]:
cnn.save("CNN_PHOSTOS_3.h5")